In [60]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math
import matplotlib.pyplot as plt

from IPython.display import clear_output

from io import StringIO
import re

import pickle
import os

from multiprocessing import Pool 
from itertools import product
import sys

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [3]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 50

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]

mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(mostPopularArtists.head())
# fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]
# print(fileNamesDFAll.head())



fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])

In [16]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF['filename'])]

featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']
featuresDF['features'] = pd.Series()

In [17]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [18]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF, bands

    return imageDF

In [19]:
def pickleDF(fileName,dataFrame,numSplits):
    print(numSplits)
    dfArray = np.array_split(dataFrame, numSplits)
    for i, df in enumerate(dfArray):
        df.to_pickle(fileName+"-"+str(i))

In [61]:
def getImgData(dataFrame,baseName):
    index = 0
    while os.path.exists(baseName+"-"+str(index)):
        index = index + 1
    
    newFileName = baseName + "-" + str(index) + ".pckl"
    print(newFileName)
    sys.stdout.flush()
    for row in tqdm(list(dataFrame.index)):
        featureSeries = pd.Series()
        if (type(dataFrame.loc[row]['features']) != str and type(dataFrame.loc[row]['features']) != pd.core.frame.DataFrame):
            imageFeatures = featurizeImage(dataFrame.loc[row]['new_filename'])
            featureSeries.append(imageFeatures)
        sys.stdout.flush()
        
    dataFrame['features'] = featureSeries
    dataFrame.to_pickle(newfileName)
    return dataFrame

In [62]:
baseFileName = 'trainImgsAllcolorBands.pckl'

splitFeaturesData = np.array_split(featuresDF,50)
i = 0
for splitFrame in splitFeaturesData:
    transformedFrame = getImgData(splitFrame,baseFileName)
    print(transformedFrame.head())
    i = i+1

# loadFile = False
# loadMore = True

# if os.path.exists(fileName) and loadFile:
#     imageFeaturesDF = pd.read_pickle(fileName)
# else:
#     imageFeaturesDF = pd.DataFrame(index=featuresDF.index,columns=['features']) 


# if loadMore:
#     i = 0
#     for row in tqdm(list(featuresDF.index)):
#         i = i + 1
#         if (type(imageFeaturesDF.loc[row]['features']) != str and type(imageFeaturesDF.loc[row]['features']) != pd.core.frame.DataFrame):
#             imageFeatures = featurizeImage(featuresDF.loc[row]['new_filename'])
#             imageFeaturesDF.loc[row]['features'] = imageFeatures
#             if i % 1 == 0:
#                 print(row)
#                 pickleDF(fileName,imageFeaturesDF,i%50)
#                 break
#     imageFeaturesDF.to_pickle(fileName)


trainImgsAllcolorBands.pckl-1


KeyboardInterrupt: 

In [ ]:
baseFileName = 'trainImgsAllcolorBands.pckl'
pool = Pool()
pool.starmap(getImgData, product(splitFeaturesData, baseFileName))
pool.close() 
pool.join()

n-0
s-0
c-0
t-0


/Users/adriencogny/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/adriencogny/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/adriencogny/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/adriencogny/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
